# Set train date range

In [85]:
STARTDATE = '2024-01-02'
ENDDATE = '2024-09-30'

# Set test date range

In [86]:
STARTDATE_test = '2024-10-08'
ENDDATE_test = '2024-12-31'

# Steps

1. Use k means cluster to cluster the stock date base on the 'processed_data_final.csv'
2. Use kmc labeled data 'clustered.csv' to train svc/ovo

In [87]:
import global_resources as gr
import os

In [88]:
data_path = os.path.join(gr.global_paths['Data'], 'Processed Data Cache', 'clustered.csv')
df = gr.read_and_return_pd_df(data_path)
df['证券代码'] = df['证券代码'].astype(str).str.zfill(6)
stock_id_df = df['证券代码']
df.drop(['证券代码'], axis = 1, inplace = True)
display(df)


Reading files from: D:\ImportanFiles\Coding Related\Repositories\Quantitative-Investment-Algorithms\Data\Processed Data Cache\clustered.csv


,Label,营业总收入,现金资产比率,应收类资产比率,固定资产比率,无形资产比率,金融负债比率,少数股东权益占比,市盈率（PE）1,资产负债率,市值A,托宾Q值A,账面市值比A,资产报酬率A,净资产收益率（ROE）A
0,4,1.294663,-0.777553,-1.270996,-1.297656,-0.692415,-1.477725,-0.185208,-0.330143,2.323068,4.379345,-0.733083,1.421790,-0.375351,0.132565
1,4,3.324778,-0.706152,-1.208916,-1.205924,-0.608636,-0.188667,1.208350,-0.326349,1.446345,0.858528,-0.829444,1.958758,-0.737222,-0.103471
2,6,-0.151864,0.133231,2.573513,-1.277036,-0.136708,-1.085169,-0.257693,-0.370844,0.840408,-0.068671,3.617179,-2.012428,-2.865983,-0.471754
3,4,-0.076711,-0.165453,-1.251412,-1.287878,-0.710005,0.400996,0.012098,-0.370844,1.215475,-0.053009,-0.664129,1.108182,-1.008012,-0.196261
4,6,-0.150626,1.671150,-0.842090,-0.384225,-0.707361,0.594283,0.100501,0.064576,0.780991,-0.068667,4.447951,-2.103008,-0.356346,0.001572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,2,-0.148916,0.255536,0.050418,0.063653,-0.494292,-1.182638,-0.185208,-0.140737,-1.544378,-0.069825,-0.522023,0.592871,0.969871,0.106275
5317,0,-0.149857,0.215799,3.219931,-0.949145,-0.670456,0.700225,-0.185208,-0.148138,-0.198913,-0.069806,0.143840,-0.678933,1.255378,0.184321
5318,7,-0.150593,0.936079,-0.717003,0.952690,0.067164,1.518683,-0.185208,-0.019689,0.223056,-0.069697,-0.208876,-0.160995,0.756657,0.153632
5319,5,-0.148657,1.784303,-0.655317,-1.041496,0.039194,-0.601230,-0.185208,-0.230032,0.294809,-0.069054,-0.508312,0.550499,0.700999,0.165490


In [89]:
labels_uni = df['Label'].unique()

In [90]:
# build a dict mapping the stringified label to the sub‑DataFrame
dfs = {
    str(lbl): df[df['Label'] == lbl].copy()
    for lbl in labels_uni
}

In [91]:
keys = list(dfs.keys())

# OVO

In [92]:
import SVM.SVC as svc
import K_means_cluster_module.k_means_cluster as kmc
import torch
import pandas as pd
import numpy as np
import random

In [93]:
device = gr.set_device()
print(f"Running on {device}.")

Running on cuda.


# testing on label 0 and 1

In [94]:
feature_cols = [c for c in df.columns if c != 'Label']
X_all = torch.tensor(df[feature_cols].values,
                     dtype=torch.float32,
                     device=device)   # [N, D]
y_all = torch.tensor(df['Label'].values,
                     dtype=torch.long,
                     device=device)  # [N]

In [95]:
display(X_all.shape, y_all.shape)

torch.Size([5321, 14])

torch.Size([5321])

In [96]:
weights_bias = svc.create_random_weights_bias(X_all.shape[-1] + 1, dtype = X_all.dtype)

Creating random weights and bias with dtype: torch.float32


In [97]:
# 3) Pick two random labels (for illustration) and train just that one binary SVM:
labels = y_all.unique().tolist()
i, j = random.sample(labels, 2)   # e.g. (3, 7)
print(labels)
print(i, j)
# 3a) Mask-out only the samples belonging to classes i or j
mask_ij = (y_all == i) | (y_all == j)
X_ij = X_all[mask_ij]              # [N_ij, D]
y_ij = y_all[mask_ij]              # [N_ij]

# 3b) Relabel i → +1, j → -1 all on-device
y_bin = torch.where(
    y_ij == i,
    torch.tensor(1, dtype=torch.float32, device=device),
    torch.tensor(-1, dtype=torch.float32, device=device)
)                                  # [N_ij], type float for hinge loss

# 3c) Call your train() on exactly this subset
w_ij, b_ij = svc.train(
    X = X_ij,
    y = y_bin,
    weights = weights_bias[:-1],    # your init
    bias    = weights_bias[-1],
    l2_penalty = True,
    print_every = 100,
    patience = 10
)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
4 7
Training with loss function: hinge loss with l2 penalty on weights.
Epoch 0 | Loss: 1.1026160717010498 | Delta loss: None
Epoch 1 | Loss: 1.0864309072494507 | Delta loss: 0.01618516445159912
Epoch 100 | Loss: 0.06505852192640305 | Delta loss: 0.001131601631641388
Epoch 200 | Loss: 0.03590237721800804 | Delta loss: 0.00010639801621437073
Epoch 300 | Loss: 0.02954377979040146 | Delta loss: 4.2105093598365784e-05
Epoch 400 | Loss: 0.02583920583128929 | Delta loss: 3.380700945854187e-05
Epoch 500 | Loss: 0.023191217333078384 | Delta loss: 2.385862171649933e-05
Epoch 600 | Loss: 0.021468954160809517 | Delta loss: 1.560710370540619e-05
Epoch 700 | Loss: 0.020146504044532776 | Delta loss: 1.7276033759117126e-05
Epoch 800 | Loss: 0.01896018348634243 | Delta loss: 1.3696029782295227e-05
Epoch 900 | Loss: 0.01796347089111805 | Delta loss: 7.072463631629944e-06
Epoch 1000 | Loss: 0.01709677465260029 | Delta loss: 7.729977369308472e-06
Epoch 1100 | Loss: 0.016353

In [98]:
accuracy = svc.score(w_ij, b_ij, X_ij, y_bin)
print(f"Testing Completed with the accuracy of {accuracy}. ")

Testing Completed with the accuracy of 99.88186650915534. 
